Prueba

In [2]:
import streamlit as st
import pandas as pd
from gensim import corpora
from gensim.models import LdaModel
from gensim.parsing.preprocessing import STOPWORDS
from gensim.parsing.porter import PorterStemmer

In [6]:
# Cargar el DataFrame
df = pd.read_csv("..\datasets\csv\sentiment_analysis.csv")
df.drop('Unnamed: 0',axis=1, inplace=True)

In [4]:
# Preprocesamiento de texto
stopwords = set(STOPWORDS)
porter_stemmer = PorterStemmer()

def preprocess_text(text):
    # Agregar aquí cualquier otro preprocesamiento que desees aplicar
    processed_text = [porter_stemmer.stem(word) for word in text.lower().split() if word not in stopwords]
    return processed_text

In [8]:
#Aplicar preprocesamiento al texto
df['processed_text'] = df['text'].apply(preprocess_text)

In [9]:
# Crear un diccionario y corpus para el modelo LDA
dictionary = corpora.Dictionary(df['processed_text'])
corpus = [dictionary.doc2bow(text) for text in df['processed_text']]

# Entrenar el modelo LDA
lda_model = LdaModel(corpus, num_topics=5, id2word=dictionary)

In [10]:
# Crear filtros interactivos en Streamlit
st.sidebar.title("Filtros")
selected_business_id = st.sidebar.selectbox("Seleccionar business_id:", df['business_id'].unique())
selected_stars = st.sidebar.selectbox("Seleccionar calificación:", sorted(df['stars'].unique()))



2024-01-16 09:57:19.646 
  command:

    streamlit run C:\Users\Grethel\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\ipykernel_launcher.py [ARGUMENTS]


In [ ]:
# Filtrar el DataFrame según los filtros seleccionados
filtered_df = df[(df['business_id'] == selected_business_id) & (df['stars'] == selected_stars)]

# Mostrar los 5 temas resultantes
st.title("Top 5 Temas según Filtros")
for index, row in filtered_df.head(5).iterrows():
    st.write(f"**Tema para el comentario #{index + 1}:**")
    st.write(lda_model[dictionary.doc2bow(row['processed_text'])])
    st.write(f"**Texto del comentario:** {row['text']}")
    st.write("---")